# ML4NLP1
## Starting Point for Exercise 1, part II

This notebook is supposed to serve as a starting point and/or inspiration when starting exercise 1, part II.

One of the goals of this exercise is o make you acquainted with **skorch**. You will probably need to consult the [documentation](https://skorch.readthedocs.io/en/stable/).

# Installing skorch and loading libraries

In [3]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch'])
except ImportError:
    pass

In [4]:
import torch
from torch import nn
import torch.nn.functional as F
from skorch import NeuralNetClassifier

In [5]:
torch.manual_seed(0)
torch.cuda.manual_seed(0)

In [6]:
import pandas as pd
import numpy as np
import csv
import re
import string
from collections import defaultdict

## Training a classifier and making predictions

In [8]:
# download dataset
!gdown 1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs # x_train
!gdown 1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6 # x_test
!gdown 1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl # y_train
!gdown 1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X # y_test

Downloading...
From: https://drive.google.com/uc?id=1QP6YuwdKFNUPpvhOaAcvv2Pcp4JMbIRs
To: /content/x_train.txt
100% 64.1M/64.1M [00:01<00:00, 39.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QVo7PZAdiZKzifK8kwhEr_umosiDCUx6
To: /content/x_test.txt
100% 65.2M/65.2M [00:00<00:00, 186MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QbBeKcmG2ZyAEFB3AKGTgSWQ1YEMn2jl
To: /content/y_train.txt
100% 480k/480k [00:00<00:00, 18.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QaZj6bI7_78ymnN8IpSk4gVvg-C9fA6X
To: /content/y_test.txt
100% 480k/480k [00:00<00:00, 18.0MB/s]


In [9]:
with open(f'x_train.txt') as f:
    x_train = f.read().splitlines()
with open(f'y_train.txt') as f:
    y_train = f.read().splitlines()
with open(f'x_test.txt') as f:
    x_test = f.read().splitlines()
with open(f'y_test.txt') as f:
    y_test = f.read().splitlines()

In [10]:
import pandas as pd
# combine x_train and y_train into one dataframe
train_df = pd.DataFrame({'text': x_train, 'label': y_train})

#combine x_test and y_test into one dataframe
test_df = pd.DataFrame({'text': x_test, 'label': y_test})

In [11]:
# T: Please use again the train/test data that includes English, German, Dutch, Danish, Swedish and Norwegian, plus 20 additional languages of your choice (the labels can be found in the file labels.csv)
# and adjust the train/test split if needed
from sklearn.model_selection import train_test_split
all_df = pd.DataFrame({'text': x_train + x_test, 'label': y_train + y_test})
train_df_re, test_df_re = train_test_split(all_df, test_size=0.2, stratify=all_df['label'])
train_df_re, valid_df = train_test_split(train_df_re, test_size=0.2, stratify=train_df_re['label'])

In [12]:
#lables
decided_labels = ['eng', 'deu', 'nld', 'dan', 'swe', 'nno', 'jpn']

additional_labels = ['lzh', 'kor', 'ell', 'fra', 'isl',
                     'ita', 'pol', 'spa', 'rus', 'ara',
                     'ang', 'fas', 'lat', 'por', 'hye',
                     'tur', 'chr', 'ind',  'zea', 'hat']

selected_labels = decided_labels + additional_labels

train_subset = train_df_re[train_df_re['label'].isin(selected_labels)]
valid_subset = valid_df[valid_df['label'].isin(selected_labels)]
test_subset = test_df_re[test_df_re['label'].isin(selected_labels)]

In [13]:
train_subset = train_subset.copy()
valid_subset = valid_subset.copy()
test_subset = test_subset.copy()

In [14]:
#Clean text
train_subset['text'] = train_subset['text'].str.lower()
valid_subset['text'] = valid_subset['text'].str.lower()
test_subset['text'] = test_subset['text'].str.lower()

In [15]:
#Clean text
def remove_punctuation_and_numbers(text):
    return re.sub(r'[^\w\s]|_', '', text)

train_subset['text'] = train_subset['text'].apply(remove_punctuation_and_numbers)
valid_subset['text'] = valid_subset['text'].apply(remove_punctuation_and_numbers)
test_subset['text'] = test_subset['text'].apply(remove_punctuation_and_numbers)

In [16]:
#Label encode
from sklearn.preprocessing import LabelEncoder
le_fitted = LabelEncoder().fit(train_subset['label'])

y_train_subset = le_fitted.transform(train_subset['label'])
y_valid_subset = le_fitted.transform(valid_subset['label'])
y_test_subset = le_fitted.transform(test_subset['label'])

Original version

In [1]:
# T: In the following, you can find a small (almost) working example of a neural network. Unfortunately, again, the cat messed up some of the code. Please fix the code such that it is executable.

In [17]:
# First, we extract some simple features as input for the neural network

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,2), max_features=100, binary=True)
X = vectorizer.fit_transform(train_subset['text'].to_numpy())

In [18]:
# Validation dataset
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
X_valid = vectorizer.transform(valid_subset['text'].to_numpy())
X_valid = X_valid.astype(np.float32)
y_valid = y_valid_subset.astype(np.int64)
valid_ds = Dataset(X_valid, y_valid)

In [19]:
X = X.astype(np.float32)
y = y_train_subset.astype(np.int64)

In the following, we define a vanilla neural network with two hidden layers. The output layer should have as many outputs as there are classes. In addition, it should have a nonlinearity function.

In [20]:
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
            num_classes=27
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units

        self.dense0 = nn.Linear(100, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, num_classes)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [21]:
X = X.astype(np.float32)
y = y_train_subset.astype(np.int64)

In [22]:
# Use CPU
net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    train_split=predefined_split(valid_ds),
    device = 'cuda' if torch.cuda.is_available() else 'cpu' # comment this to train with CPU
)

In [23]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        3.0073       0.2785        2.6321  1.9663
      2        2.1372       0.4315        1.7380  1.7973
      3        1.4961       0.5338        1.3481  2.2392
      4        1.2529       0.5745        1.1875  2.9329
      5        1.1380       0.6007        1.1044  3.3591
      6        1.0712       0.6322        1.0563  1.8145
      7        1.0253       0.6410        1.0235  1.7978
      8        0.9895       0.6486        0.9984  1.7899
      9        0.9608       0.6528        0.9810  1.7985
     10        0.9387       0.6551        0.9694  1.7577
     11        0.9220       0.6553        0.9608  2.3324
     12        0.9085       0.6574        0.9553  1.8234
     13        0.8972       0.6512        0.9515  1.7411
     14        0.8874       0.6493        0.9483  1.7689
     15        0.8790       0.6488        0.9460  1.7690
     16        0.8715       0.6

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=100, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=27, bias=True)
  ),
)

Max_features_changed

In [ ]:
# Changed the max_features to 1000
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(analyzer='char', ngram_range=(2,2), max_features=1000, binary=True)
X = vectorizer.fit_transform(train_subset['text'].to_numpy())

In [ ]:
# Validation dataset
import skorch
from skorch.helper import predefined_split
from skorch.dataset import Dataset
X_valid = vectorizer.transform(valid_subset['text'].to_numpy())
X_valid = X_valid.astype(np.float32)
y_valid = y_valid_subset.astype(np.int64)
valid_ds = Dataset(X_valid, y_valid)

In [ ]:
X = X.astype(np.float32)
y = y_train_subset.astype(np.int64)

In [ ]:
#Check shape
print(X.shape)

(21600, 100)


In [ ]:
#Check num_classes
print(np.unique(y))
n_classes = len(np.unique(y))

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26]


In [ ]:
#Changed the num_classes, and self.dense0 == nn.Linear(100, num_units) based on the changing of max_features
class ClassifierModule(nn.Module):
    def __init__(
            self,
            num_units=200,
            nonlin=F.relu,
            num_classes=27
    ):
        super(ClassifierModule, self).__init__()
        self.num_units = num_units

        self.dense0 = nn.Linear(1000, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 50)
        self.output = nn.Linear(50, num_classes)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [ ]:

net = NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    train_split=predefined_split(valid_ds),
    device = 'cuda' if torch.cuda.is_available() else 'cpu' # comment this to train with CPU
)

In [ ]:
net.fit(X, y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.6837       0.6813        1.4990  3.6681
      2        0.8702       0.8977        0.4790  5.0055
      3        0.3489       0.9551        0.2761  4.4568
      4        0.2234       0.9616        0.2184  3.4463
      5        0.1766       0.9641        0.1904  3.2655
      6        0.1484       0.9630        0.1723  5.6223
      7        0.1274       0.9644        0.1599  3.6078
      8        0.1116       0.9644        0.1517  3.4585
      9        0.0995       0.9650        0.1464  3.4328
     10        0.0901       0.9667        0.1428  4.4043
     11        0.0824       0.9669        0.1405  3.2564
     12        0.0759       0.9676        0.1390  3.4075
     13        0.0702       0.9678        0.1381  4.1798
     14        0.0652       0.9676        0.1377  3.5700
     15        0.0606       0.9676        0.1377  3.2655
     16        0.0565       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=1000, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=27, bias=True)
  ),
)

In [ ]:
#Weights Initialization
def weights_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)

In [ ]:
net = skorch.NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    train_split=predefined_split(valid_ds),
    device='cuda' if torch.cuda.is_available() else 'cpu',
    callbacks=[('weights_init', skorch.callbacks.Initializer('*.*', fn=weights_init))]
    )

In [ ]:
net.fit(X,y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.7115       0.7461        1.4827  4.2845
      2        0.8410       0.8831        0.4673  3.3845
      3        0.3558       0.9556        0.2798  3.5473
      4        0.2285       0.9576        0.2188  4.5250
      5        0.1777       0.9600        0.1905  3.3483
      6        0.1486       0.9616        0.1725  3.3457
      7        0.1280       0.9625        0.1599  4.2898
      8        0.1127       0.9632        0.1509  3.5599
      9        0.1009       0.9634        0.1445  3.4065
     10        0.0915       0.9641        0.1396  3.3097
     11        0.0838       0.9655        0.1360  4.3939
     12        0.0771       0.9664        0.1334  3.4396
     13        0.0713       0.9671        0.1315  3.5226
     14        0.0662       0.9671        0.1303  5.3196
     15        0.0616       0.9674        0.1297  4.7374
     16        0.0574       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=1000, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=27, bias=True)
  ),
)

In [ ]:
# Add earlystop
from skorch.callbacks import EarlyStopping
net = skorch.NeuralNetClassifier(
    ClassifierModule,
    max_epochs=20,
    criterion=nn.CrossEntropyLoss(),
    lr=0.1,
    train_split=predefined_split(valid_ds),
    device='cuda' if torch.cuda.is_available() else 'cpu',
    callbacks=[
        ('weights_init', skorch.callbacks.Initializer('*.*', fn=weights_init)),
        ('early_stopping', EarlyStopping(patience=5))
    ],
    verbose=0
)

In [ ]:
#Gridsearch
from sklearn.model_selection import GridSearchCV
param_grid = {
    'module__num_units': [100, 200, 300],  # Layer Sizes
    'module__nonlin': [F.relu, torch.sigmoid],  # Activation Functions
    'lr': [1, 0.1],  # Learning Rate
    'max_epochs': [10, 20],  # Epochs
    'optimizer': [torch.optim.SGD, torch.optim.Adam],# Optimizers

}

gs = GridSearchCV(net, param_grid, refit=True, cv=3, scoring='accuracy')
gs.fit(X, y)

print("Best parameters set found on development set:")
print(gs.best_params_)


Best parameters set found on development set:
{'lr': 0.1, 'max_epochs': 20, 'module__nonlin': <function relu at 0x7a6a85279b40>, 'module__num_units': 200, 'optimizer': <class 'torch.optim.sgd.SGD'>}


In [ ]:
# Update parameters
net = skorch.NeuralNetClassifier(
    ClassifierModule,

    criterion=nn.CrossEntropyLoss(),

    module__num_units=200,
    module__nonlin=F.relu,
    lr=0.1,
    max_epochs=20,
    optimizer=torch.optim.SGD,

    train_split=predefined_split(valid_ds),
    device='cuda' if torch.cuda.is_available() else 'cpu',
    callbacks=[
        ('weights_init', skorch.callbacks.Initializer('*.*', fn=weights_init)),
        ('early_stopping', EarlyStopping(patience=5))
    ]
)


In [ ]:
net.fit(X,y)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        2.7510       0.6664        1.6341  3.5182
      2        0.9239       0.8609        0.5146  3.7639
      3        0.3751       0.9512        0.2975  3.9592
      4        0.2393       0.9535        0.2326  3.4345
      5        0.1848       0.9574        0.2008  3.4058
      6        0.1539       0.9588        0.1806  4.3953
      7        0.1320       0.9597        0.1666  3.4007
      8        0.1156       0.9618        0.1571  3.3969
      9        0.1031       0.9632        0.1505  3.8815
     10        0.0933       0.9639        0.1457  4.0868
     11        0.0853       0.9646        0.1421  3.2466
     12        0.0784       0.9648        0.1392  3.2905
     13        0.0724       0.9655        0.1371  4.4577
     14        0.0670       0.9650        0.1355  3.4395
     15        0.0622       0.9657        0.1345  3.4119
     16        0.0578       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=ClassifierModule(
    (dense0): Linear(in_features=1000, out_features=200, bias=True)
    (dense1): Linear(in_features=200, out_features=50, bias=True)
    (output): Linear(in_features=50, out_features=27, bias=True)
  ),
)

In [ ]:
# Evaluate the result
X_train = vectorizer.transform(train_subset['text'].to_numpy()).astype(np.float32)
X_test = vectorizer.transform(test_subset['text'].to_numpy()).astype(np.float32)

In [ ]:
train_accuracy = net.score(X_train, y_train_subset)
valid_accuracy = net.score(X_valid, y_valid_subset)
test_accuracy = net.score(X_test, y_test_subset)

print(f"Training Accuracy: {train_accuracy*100:.2f}%")
print(f"Validation Accuracy: {valid_accuracy*100:.2f}%")
print(f"Test Accuracy: {test_accuracy*100:.2f}%")

Training Accuracy: 98.99%
Validation Accuracy: 96.50%
Test Accuracy: 96.41%
